In [140]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az
from sklearn.linear_model import LinearRegression
from typing import Dict, Optional, Tuple, Sequence, Iterator

In [22]:
"""
Import data
"""
raw_data = pd.read_pickle("data.pkl")
raw_data[0:5]

,subject,exp,phasename,num_rep,num_trial,num_formed,num_uttered,prompt,in_or_out,date_run,time_run,block,pitches
0,20,PU,5,44,1,2637,70,HECK,O,20170127,11,3,0 134.355450 1 134.360625 2 134...
1,20,PU,5,36,1,2629,62,HECK,O,20170127,11,3,0 130.217779 1 130.179978 2 130...
2,20,PU,7,20,1,2713,146,HECK,I,20170127,11,3,0 132.486939 1 132.479573 2 132...
3,20,PU,6,6,1,2649,82,HECK,O,20170127,11,3,0 139.037150 1 139.055242 2 139...
4,20,PU,7,8,1,2701,134,HECK,I,20170127,11,3,0 130.927041 1 130.913059 2 130...


In [23]:
"""
Preprocessing
"""

# Create new data frame

# Remove "O" data
df = raw_data[raw_data.in_or_out == "O"]

# Remove first block
df = df[df.block != 1]

# Rename "phasename" to "phasenum"
df.rename(columns = {"phasename": "phasenum", "exp": "condition"}, inplace = True)

# Reset pd indexes
df.reset_index()

# Preview
df[0:5]

,subject,condition,phasenum,num_rep,num_trial,num_formed,num_uttered,prompt,in_or_out,date_run,time_run,block,pitches
0,20,PU,5,44,1,2637,70,HECK,O,20170127,11,3,0 134.355450 1 134.360625 2 134...
1,20,PU,5,36,1,2629,62,HECK,O,20170127,11,3,0 130.217779 1 130.179978 2 130...
3,20,PU,6,6,1,2649,82,HECK,O,20170127,11,3,0 139.037150 1 139.055242 2 139...
5,20,PU,1,1,1,2568,1,HECK,O,20170127,11,3,0 112.625662 1 112.649338 2 112...
7,20,PU,5,39,1,2632,65,HECK,O,20170127,11,3,0 139.263470 1 139.266789 2 139...


In [24]:
"""
Generate summary statistics for each trial
"""

agg: Dict[int, float] = {
    "count": [],
    "avg": [],
    "std": [],
    "minimum": [],
    "med": [],
    "maximum": [],
}
    
for pitches in df.pitches:
    (
        count,
        avg,
        std,
        minimum,
        _,
        med,
        _,
        maximum,
                ) = pitches.describe()
    agg["count"].append(count)
    agg["avg"].append(avg)
    agg["std"].append(std)
    agg["minimum"].append(minimum)
    agg["med"].append(med)
    agg["maximum"].append(maximum)
    
# Preview summary statistics
summary_stats = pd.DataFrame(agg)
summary_stats[0:5]

,count,avg,std,minimum,med,maximum
0,491.0,133.610873,1.544445,124.930762,133.915850,134.887327
1,531.0,128.870786,1.549235,124.541467,129.371750,132.296456
2,461.0,137.901782,0.786993,135.052405,137.730314,140.124944
3,141.0,115.535289,2.270588,112.625662,114.953949,121.897112
4,451.0,134.375365,2.289648,127.755729,133.575767,139.429389


In [25]:
"""
Relativize to F0 of start phase
"""

subjects = df.subject.unique()
conditions = df.condition.unique()

# Subset only start phase trials with the prompt "HECK"
selection = ((df.phasenum == 4) & (df.prompt == "HECK")).to_numpy()
trial_f0 = summary_stats.avg[selection]
trial_subject = df.subject[selection] 
trial_condition = df.condition[selection] 

# Get the baseline F0 for each subject
f0s = {}
for subject in subjects:
    for condition in conditions:
        selection = ((subject == trial_subject) & (condition == trial_condition)).to_numpy()
        trials = trial_f0[selection]
        f0 = trials.mean()
        f0s[(subject, condition)] = f0

# Subtract pitches from baseline F0 
corrected_pitches = []

# For each trial
for i, trial_pitches in enumerate(df.pitches):
        
    # Get the subject's f0
    subject = df.subject.to_numpy()[i]
    condition = df.condition.to_numpy()[i]
    f0 = f0s[(subject, condition)]
    
    # Subtract f0 from values
    trial_corrected_pitches = trial_pitches - f0
    
    # Take mean
    trial_corrected_pitches = trial_corrected_pitches.mean()
    
    # Save in new arrage
    corrected_pitches.append(trial_corrected_pitches)
    
cleaned_df = df.drop(columns = ['pitches', 'num_trial', 'in_or_out'])
cleaned_df['corrected_pitches'] = corrected_pitches

In [28]:
print(cleaned_df)

       subject condition  phasenum  num_rep  num_formed  num_uttered prompt  \
0           20        PU         5       44        2637           70   HECK   
1           20        PU         5       36        2629           62   HECK   
3           20        PU         6        6        2649           82   HECK   
5           20        PU         1        1        2568            1   HECK   
7           20        PU         5       39        2632           65   HECK   
...        ...       ...       ...      ...         ...          ...    ...   
43780       32       FAE         4       16         777           22   HECK   
43783       32       FAE         6       29         860          105   HECK   
43784       32       FAE         6       30         861          106   HECK   
43785       32       FAE         5        9         790           35   HECK   
43787       32       FAE         4       11         772           17   HECK   

       date_run time_run  block  corrected_pitches 

In [30]:
"""
Helper functions
"""

def subjects(df) -> Sequence[int]:
    return list(df.subject.unique())

PHASENUMS = [i for i in range(4, 8)]
CONDITIONS = ["PU", "PD", "FAE", "FIH"]
SUBJECTS = subjects(cleaned_df)

phasenames = {
    "2": "pract2",
    "3": "unk",
    "4": "start",
    "5": "ramp_up",
    "6": "stay",
    "7": "ramp_down",
    }
    
def individual_keys() -> Iterator[KeyType]:
    for condition in CONDITIONS:
        for phasenum in PHASENUMS:
            for subject in SUBJECTS:
                key = (condition, phasenum, subject)
                yield key
                
def group_keys() -> Iterator[KeyType]:
    for condition in CONDITIONS:
        for phasenum in PHASENUMS:
            key = (condition, phasenum)
            yield key

def subset_data(
    df: pd.DataFrame,
    *,
    condition: Optional[str] = None,
    phasenum: Optional[int] = None,
    subject: Optional[int] = None,
) -> pd.DataFrame:
    
    if condition is not None:
        df = df[df.condition == condition]
    
    if phasenum is not None:
        df = df[df.phasenum == phasenum]
    
    if subject is not None:
        df = df[df.subject == subject]
    
    return df

In [ ]:
"""
Scatterplots for reps and change in pitch
"""

def plot_scatter(df: pd.DataFrame, phasenames: dict):
    # Get phasename
    phasename = phasenames[str(phasenum)]

    # Plot
    x = df.num_rep
    y = df.corrected_pitches
    colors = df.subject
    plt.scatter(x, y, c = colors)
    plt.title(phasename + " phase for " + condition)
    plt.xlabel("Rep")
    plt.ylabel("Change in F0")
    plt.ylim([-100, 100])
    plt.savefig("figs/" + phasename + "_" + condition + ".png")
    plt.close()

for key in group_keys():
    condition, phasenum = key
    plot_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    plot_scatter(plot_df, phasenames)

In [79]:
"""
Linear regression between rep and change in pitch for individual subjects
"""

subjects = cleaned_df.subject.unique()

# Make data frame to fill

def lin_reg(df: pd.DataFrame, phasenames: dict) -> Tuple[float, float]:
    # Get phasename
    phasename = phasenames[str(phasenum)]
    
    # Lin reg
    x = df.num_rep.to_numpy().reshape(-1, 1)
    y = df.corrected_pitches
    y.fillna(y.mean(), inplace = True)
    y = y.to_numpy().reshape(-1, 1)
    if x.shape[0] == 0 and y.shape[0] == 0:
        print("No data for", (condition, phasenum, subject))
        return None
    ln = LinearRegression().fit(x, y)
    a = ln.intercept_[0]
    b = ln.coef_[0][0]
    return a, b

subject_regressions = []

for key in individual_keys():
    condition, phasenum, subject = key
    lin_reg_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum, subject = subject)
    regression = lin_reg(lin_reg_df, phasenames)
    if regression is None:
        continue
    a, b = regression
    subject_regressions.append((phasenum, condition, subject, a, b))

No data for ('FIH', 7, 16)


In [80]:
"""
Linear regression between rep and change in pitch over all subjects
"""
group_regressions = []

for key in group_keys():
    condition, phasenum = key
    lin_reg_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    regression = lin_reg(lin_reg_df, phasenames)
    if regression is None:
        continue
    a, b = regression
    group_regressions.append((phasenum, condition, subject, a, b))

In [168]:
KeyType = Tuple[int, int, str]
ValueType = pd.DataFrame
AggBayesType = Dict[KeyType, ValueType]

agg_bayes: AggBayesType = {}

for key in group_keys():
    print("Fitting linear model for:", key)
    
    # Get data for model
    condition, phasenum = key
    test_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    x = test_df.num_rep.to_numpy().reshape(-1, 1)
    x = x.flatten()
    y = test_df.corrected_pitches.to_numpy()
    
    # Model
    with pm.Model() as model:
        # Define priors
        a = pm.Normal('slope', 0, 16)
        b = pm.Normal('intercept', 0, 16)
        s = pm.Exponential('error', 1)

        # Predictions
        obs = pm.Normal('observation', a * x + b, s, observed = y)

        # Use MCMC to sample
        trace = pm.sample(1000, tune=1000, cores=1)

        # Get summary statistics for posterior
        lin_reg_summary = az.summary(trace, round_to = 2, hdi_prob = 0.95)
        agg_bayes[key] = lin_reg_summary.loc[["slope", "intercept", "error"], 
                                            ["mean", "sd", "hdi_2.5%", "hdi_97.5%"]]


Fitting linear model for: ('PU', 4)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
The acceptance probability does not match the target. It is 0.8888963114891927, but should be close to 0.8. Try to increase the number of tuning steps.


Fitting linear model for: ('PU', 5)


/opt/anaconda3/lib/python3.7/site-packages/pymc3/model.py:1755: ImputationWarning: Data in observation contains missing values and will be automatically imputed from the sampling distribution.
  if self.distribution is None:
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [observation_missing, error, intercept, slope]


...

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.


Fitting linear model for: ('FIH', 7)


/opt/anaconda3/lib/python3.7/site-packages/pymc3/model.py:1755: ImputationWarning: Data in observation contains missing values and will be automatically imputed from the sampling distribution.
  if self.distribution is None:
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [observation_missing, error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 8 seconds.
The acceptance probability does not match the target. It is 0.8913314039676368, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8804208844736539, but should be close to 0.8. Try to increase the number of tuning steps.


In [ ]:
#agg_bayes: Dict[tuple, float] = {
#    "avg": [],
#    "std": [],
#    "hdi_upper": [],
#    "hdi_lower": [],
#}
#indexes = []

for key in group_keys():
    print("Fitting linear model for:", key)
    
    # Get data for model
    condition, phasenum = key
    test_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    x = test_df.num_rep.to_numpy().reshape(-1, 1)
    x = x.flatten()
    y = test_df.corrected_pitches.to_numpy()
    
    # Model
    with pm.Model() as model:
        # Define priors
        a = pm.Normal('slope', 0, 16)
        b = pm.Normal('intercept', 0, 16)
        s = pm.Exponential('error', 1)

        # Predictions
        obs = pm.Normal('observation', a * x + b, s, observed = y)

        # Use MCMC to sample
        trace = pm.sample(1000, tune=1000, cores=1)

        # Get summary statistics for posterior
        lin_reg_summary = az.summary(trace, round_to = 2, hdi_prob = 0.95)
        agg_bayes[key] = lin_reg_summary.loc[["slope", "intercept", "error"], 
                                            ["mean", "sd", "hdi_2.5%", "hdi_97.5%"]]
        
        # Append to dict
        # = lin_reg_summary.iloc[["slope", "intercept", "error"], ["mean"]]
        #indexes.append(key)
        #agg_bayes["avg"].append(avg)
        #agg_bayes["std"].append(std)
        #agg_bayes["hdi_lower"].append(hdi_lower)
        #agg_bayes["hdi_upper"].append(hdi_upper)
        
#agg_bayes = pd.DataFrame(agg_bayes)
#agg_bayes = agg_bayes.set_index(indexes) 


In [165]:
lin_reg_summary.loc[["slope"], ["mean"]]
lin_reg_summary.iloc[0, 0]

-0.05

In [152]:
type(lin_reg_summary.loc[["slope", "intercept", "error"], ["mean", "sd", "hdi_2.5%", "hdi_97.5%"]])

pandas.core.frame.DataFrame

In [169]:
agg_bayes

{('PU', 4):            mean    sd  hdi_2.5%  hdi_97.5%
 slope     -0.05  0.04     -0.12       0.03
 intercept  0.49  0.50     -0.41       1.51
 error      6.07  0.17      5.73       6.41,
 ('PU', 5):            mean    sd  hdi_2.5%  hdi_97.5%
 slope      0.25  0.02      0.23       0.29
 intercept -0.91  0.46     -1.76       0.04
 error      8.71  0.16      8.40       9.03,
 ('PU', 6):             mean    sd  hdi_2.5%  hdi_97.5%
 slope      -0.00  0.02     -0.04       0.04
 intercept  13.42  0.65     12.21      14.71
 error      12.04  0.23     11.60      12.47,
 ('PU', 7):            mean    sd  hdi_2.5%  hdi_97.5%
 slope     -0.12  0.05     -0.22      -0.02
 intercept  4.23  0.76      2.77       5.72
 error      9.93  0.24      9.47      10.41,
 ('PD', 4):             mean    sd  hdi_2.5%  hdi_97.5%
 slope      -0.04  0.08     -0.19       0.13
 intercept   0.38  0.96     -1.46       2.27
 error      10.94  0.32     10.31      11.56,
 ('PD', 5):             mean    sd  hdi_2.5%  hdi_97

array([0.46341258])

AttributeError: 'LinearRegression' object has no attribute 'summary_'